In [12]:
import os
import re
import sys
import cv2
import tqdm
import time
import spacy 
import random
import scipy.io
import itertools
import numpy as np
from math import ceil
import pandas as pd
from itertools import chain
from tqdm.contrib import tzip
import matplotlib.pyplot as plt
from skimage.io import imread
from scipy.ndimage.filters import gaussian_filter
from sklearn.model_selection import train_test_split

In [13]:
import torch
from torch.nn.utils.rnn import pad_sequence 
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

In [14]:
pattern_repl = re.compile('(<e1>)|(</e1>)|(<e2>)|(</e2>)|(\'s)')
pattern_e1 = re.compile('<e1>(.*)</e1>')
pattern_e2 = re.compile('<e2>(.*)</e2>')
pattern_symbol = re.compile('^[!"#$%&\\\'()*+,-./:;<=>?@[\\]^_`{|}~]|[!"#$%&\\\'()*+,-./:;<=>?@[\\]^_`{|}~]$')

In [15]:
data_path = '../../datasets/part2/TRAIN_FILE.TXT'
f = open(data_path)

In [20]:
glove_ukn = [float(i) for i in '0.22418134 -0.28881392 0.13854356 0.00365387 -0.12870757 0.10243822 0.061626635 0.07318011 -0.061350107 -1.3477012 0.42037755 -0.063593924 -0.09683349 0.18086134 0.23704372 0.014126852 0.170096 -1.1491593 0.31497982 0.06622181 0.024687296 0.076693475 0.13851812 0.021302193 -0.06640582 -0.010336159 0.13523154 -0.042144544 -0.11938788 0.006948221 0.13333307 -0.18276379 0.052385733 0.008943111 -0.23957317 0.08500333 -0.006894406 0.0015864656 0.063391194 0.19177166 -0.13113557 -0.11295479 -0.14276934 0.03413971 -0.034278486 -0.051366422 0.18891625 -0.16673574 -0.057783455 0.036823478 0.08078679 0.022949161 0.033298038 0.011784158 0.05643189 -0.042776518 0.011959623 0.011552498 -0.0007971594 0.11300405 -0.031369694 -0.0061559738 -0.009043574 -0.415336 -0.18870236 0.13708843 0.005911723 -0.113035575 -0.030096142 -0.23908928 -0.05354085 -0.044904727 -0.20228513 0.0065645403 -0.09578946 -0.07391877 -0.06487607 0.111740574 -0.048649278 -0.16565254 -0.052037314 -0.078968436 0.13684988 0.0757494 -0.006275573 0.28693774 0.52017444 -0.0877165 -0.33010918 -0.1359622 0.114895485 -0.09744406 0.06269521 0.12118575 -0.08026362 0.35256687 -0.060017522 -0.04889904 -0.06828978 0.088740796 0.003964443 -0.0766291 0.1263925 0.07809314 -0.023164088 -0.5680669 -0.037892066 -0.1350967 -0.11351585 -0.111434504 -0.0905027 0.25174105 -0.14841858 0.034635577 -0.07334565 0.06320108 -0.038343467 -0.05413284 0.042197507 -0.090380974 -0.070528865 -0.009174437 0.009069661 0.1405178 0.02958134 -0.036431845 -0.08625681 0.042951006 0.08230793 0.0903314 -0.12279937 -0.013899368 0.048119213 0.08678239 -0.14450377 -0.04424887 0.018319942 0.015026873 -0.100526 0.06021201 0.74059093 -0.0016333034 -0.24960588 -0.023739101 0.016396184 0.11928964 0.13950661 -0.031624354 -0.01645025 0.14079992 -0.0002824564 -0.08052984 -0.0021310581 -0.025350995 0.086938225 0.14308536 0.17146006 -0.13943303 0.048792403 0.09274929 -0.053167373 0.031103406 0.012354865 0.21057427 0.32618305 0.18015954 -0.15881181 0.15322933 -0.22558987 -0.04200665 0.0084689725 0.038156632 0.15188617 0.13274793 0.113756925 -0.095273495 -0.049490947 -0.10265804 -0.27064866 -0.034567792 -0.018810693 -0.0010360252 0.10340131 0.13883452 0.21131058 -0.01981019 0.1833468 -0.10751636 -0.03128868 0.02518242 0.23232952 0.042052146 0.11731903 -0.15506615 0.0063580726 -0.15429358 0.1511722 0.12745973 0.2576985 -0.25486213 -0.0709463 0.17983761 0.054027 -0.09884228 -0.24595179 -0.093028545 -0.028203879 0.094398156 0.09233813 0.029291354 0.13110267 0.15682974 -0.016919162 0.23927948 -0.1343307 -0.22422817 0.14634751 -0.064993896 0.4703685 -0.027190214 0.06224946 -0.091360025 0.21490277 -0.19562101 -0.10032754 -0.09056772 -0.06203493 -0.18876675 -0.10963594 -0.27734384 0.12616494 -0.02217992 -0.16058226 -0.080475815 0.026953284 0.110732645 0.014894041 0.09416802 0.14299914 -0.1594008 -0.066080004 -0.007995227 -0.11668856 -0.13081996 -0.09237365 0.14741232 0.09180138 0.081735 0.3211204 -0.0036552632 -0.047030564 -0.02311798 0.048961394 0.08669574 -0.06766279 -0.50028914 -0.048515294 0.14144728 -0.032994404 -0.11954345 -0.14929578 -0.2388355 -0.019883996 -0.15917352 -0.052084364 0.2801028 -0.0029121689 -0.054581646 -0.47385484 0.17112483 -0.12066923 -0.042173345 0.1395337 0.26115036 0.012869649 0.009291686 -0.0026459037 -0.075331464 0.017840583 -0.26869613 -0.21820338 -0.17084768 -0.1022808 -0.055290595 0.13513643 0.12362477 -0.10980586 0.13980341 -0.20233242 0.08813751 0.3849736 -0.10653763 -0.06199595 0.028849555 0.03230154 0.023856193 0.069950655 0.19310954 -0.077677034 -0.144811'.split()]

In [16]:
class DataUtils():
    def __init__(self, data_path = data_path, min_word_frequency = 2):
        self.min_word_frequency = min_word_frequency
        self.english_tokenizer = spacy.load("en_core_web_sm")
        self.max_cap_length = 30
        
        print("loading data ...")
        self.data = self.load_dataset(data_path)
        print("---done!---")
        
        print("creating vocabulary ...")
        self.create_vocabulary(self.data)
        
        print("numericalizing sentences ...")
        self.numericalize_sentences = self.sentences_numericalizer(self.data)
        
        self.indexes = []
        asdf = 0
        for i in self.data:
            try:
                a, b = i[0][0].split()[0], i[0][1].split()[0]
                self.indexes.append((i[0][2].split().index(a), i[0][2].split().index(b)))
            except:
                a, b = i[0][0].split()[0], i[0][1].split()[0]
                if not a in i[0][2].split():
                    l = 0
                else:
                    l = i[0][2].split().index(a)
                if not b in i[0][2].split():
                    k = len(i[0][2].split())-1
                else:
                    k = i[0][2].split().index(b)
                self.indexes.append((l,k))
        
        self.all_classes = list(np.unique([i[1].split('(')[0] for i in self.data]))
        all_classes = self.all_classes
        self.classes = []
        for i in self.data:
            self.classes.append(all_classes.index(i[1].split('(')[0]))
            
        print("sentences:", len(self.data), "sentences:",  len(self.numericalize_sentences), "Vocab:", len(self.vocabulary))
        
        
    def delete_symbol(self, text):
        if pattern_symbol.search(text):
            return pattern_symbol.sub('', text)
        return text

    def load_dataset(self, path_dataset):
        dataset = []
        with open(path_dataset) as f:
            piece = []
            for line in f:
                line = line.strip()
                if line:
                    piece.append(line)
                elif piece:
                    if int(piece[0].split('\t')[0]) > 7109:
                        break
                    sentence = piece[0].split('\t')[1].strip('"')
                    e1 = self.delete_symbol(pattern_e1.findall(sentence)[0])
                    e2 = self.delete_symbol(pattern_e2.findall(sentence)[0])
                    new_sentence = list()
                    for word in pattern_repl.sub('', sentence).split(' '):
                        new_word = self.delete_symbol(word)
                        if new_word:
                            new_sentence.append(new_word)

                    relation = piece[1]
                    dataset.append(((e1, e2, ' '.join(new_sentence)), relation))
                    piece = list()
        return dataset
    
    def tokenizer_eng(self, text):
        return [tok.text.lower() for tok in self.english_tokenizer.tokenizer(text)]
    
    def create_vocabulary(self, data):
        self.vocabulary = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.rev_vocabulary = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        min_word_frequency = self.min_word_frequency
        
        frequencies = {}
        idx = 4
        
        counter, count = 1, len(data)
        for datum in data:
            sentence = datum[0][2]
            for word in self.tokenizer_eng(sentence):
                if word not in frequencies:
                    frequencies[word] = 1
                else:
                    frequencies[word] += 1

                if frequencies[word] == min_word_frequency:
                    self.rev_vocabulary[word] = idx
                    self.vocabulary[idx] = word
                    idx += 1
            counter += 1
            print(str(round(counter/count*100, 2))+'%', end="\r")
        print('---done!---')
        
    def numericalize_sentence(self, caption):
        numericalized_caption = [self.rev_vocabulary["<SOS>"]]
        numericalized_caption += self.numericalize(caption)
        numericalized_caption.append(self.rev_vocabulary["<EOS>"])
        return numericalized_caption
    
    def sentences_numericalizer(self, data):
        numericalized_sentences = []
        
        counter, count = 1, len(data)
        for datum in data:
            sentence = datum[0][2]
            
            padded_numericalized = self.pad(self.numericalize_sentence(sentence))
            numericalized_sentences.append(padded_numericalized)
            counter += 1
            print(str(round(counter/count*100, 2))+'%', end="\r")
        print("---done!---!")
        return numericalized_sentences
        
    def pad(self, sequence):
        max_cap_length = self.max_cap_length
        if len(sequence) > max_cap_length:
            sequence = sequence[:max_cap_length]
            sequence[max_cap_length-1] = self.rev_vocabulary["<EOS>"]
        else:
            while(len(sequence) < max_cap_length):
                sequence.append(self.rev_vocabulary["<PAD>"])
        return sequence
    
    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)

        return [
            self.rev_vocabulary[token] if token in self.rev_vocabulary else self.rev_vocabulary["<UNK>"]
            for token in tokenized_text
        ]
        
    def reverse_numericalize(self, encoded):

        strings = []
            
        for token in encoded:
            if token in self.vocabulary:
                strings.append(self.vocabulary[token])
            else:
                strings.append(self.vocabulary[3])
            if strings[-1] == '<EOS>':
                break
        
        
        if '<SOS>' in strings:
            strings.remove('<SOS>')
        if '<EOS>' in strings:
            strings.remove('<EOS>')
        while '<PAD>' in strings:
            strings.remove('<PAD>')
            
        sentence = ''
        for i in strings:
            sentence += i
            sentence += ' '
            
        return sentence[:-1]
    
    def load_glove(self):
        glove_vocabulary = {}
        counter, count = 1, len(self.rev_vocabulary.keys())
        with open('../../datasets/glove.42B.300d.txt') as f:
            for i in f:
                spl = i.split()
                if spl[0] in self.rev_vocabulary.keys():
                    l = [float(j) for j in spl[1:]]
                    glove_vocabulary[self.rev_vocabulary[spl[0]]] = l
                    counter += 1
                    print(str(round(counter/count*100, 2))+'%', end="\r")
                    
        glove_vocabulary[self.rev_vocabulary["<PAD>"]] = glove_vocabulary[self.rev_vocabulary["."]]
        glove_vocabulary[self.rev_vocabulary["<EOS>"]] = glove_vocabulary[self.rev_vocabulary["."]]
        glove_vocabulary[self.rev_vocabulary["<UNK>"]] = glove_ukn
        print("---done!---")
        return glove_vocabulary
    
    def get_sentence_class(self, test_size):
        self.sentence_train, self.sentence_test, self.c_trin, self.c_test =\
            train_test_split(self.numericalize_sentences, self.classes, test_size = test_size, random_state=42)
        return self.sentence_train, self.sentence_test, self.c_trin, self.c_test
    
    def get_glove_sentence_class(self, test_size):
        print("loading glove ...")
        glovee = self.load_glove()
        self.glove_vecs = []
        for i in self.numericalize_sentences:
            self. glove_vecs.append([glovee[j] if j in glovee.keys() else glovee[3] for j in i[1:]])
        self.sentence_train, self.sentence_test, self.c_trin, self.c_test =\
            train_test_split(self.glove_vecs, self.classes, test_size = test_size, random_state=42)
        return self.sentence_train, self.sentence_test, self.c_trin, self.c_test
    
    def get_glove_indexed_sentence_class(self, test_size):
        print("loading glove ...")
        glovee = self.load_glove()
        self.glove_vecs = []
        for i in self.numericalize_sentences:
            self. glove_vecs.append([glovee[j] if j in glovee.keys() else glovee[3] for j in i[1:]])
        out_data = []
        for i,j in zip(self.glove_vecs, self.indexes):
            out_data.append([i, j])
            
        self.sentence_train, self.sentence_test, self.c_trin, self.c_test =\
            train_test_split(out_data, self.classes, test_size = test_size, random_state=42)
        return self.sentence_train, self.sentence_test, self.c_trin, self.c_test
        

In [17]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sentences, classes):
        self.X, self.y = sentences, classes
    def __len__(self):
        return len(self.X)
    def __getitem__(self, index):
        classs = self.y[index]
        sentence = self.X[index]
        
        return torch.tensor(sentence), classs

In [18]:
a = DataUtils()

loading data ...
---done!---
creating vocabulary ...
---done!---
numericalizing sentences ...
---done!---!
sentences: 7109 sentences: 7109 Vocab: 8458


In [19]:
a.all_classes

['Cause-Effect',
 'Component-Whole',
 'Content-Container',
 'Entity-Destination',
 'Entity-Origin',
 'Instrument-Agency',
 'Member-Collection',
 'Message-Topic',
 'Other',
 'Product-Producer']